In [43]:
import cgi
import json
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import os

In [44]:
html=requests.get('https://www.salonecriture.org/actualités/')
soup=BeautifulSoup(html.content, 'html.parser')
soup=soup.find('div', attrs={'class': 'jtpl-content__inner content-options cc-content-parent'})

### Links

In [45]:
all_links = []
for ref in soup.findAll('a', attrs={'class': 'blogreadmore'}):
    all_links.append(ref['href'])

In [46]:
all_links

['/2017/02/12/ghostwriter-écrivain-fantôme-prête-plume-qu-est-ce/',
 '/2017/01/28/secrétaire-juridique-l-écriture-ce-vecteur-codé-et-procédural/',
 '/2017/01/25/maître-marc-bonnant-à-l-honneur/',
 '/2017/01/17/harmonie-keller-matthey-blogueuse-l-écriture-est-un-art-libérateur/',
 '/2017/01/14/exposition-sur-l-alphabet-a-ka-u-ku/',
 '/2017/01/11/rédacteur-technique-ou-l-art-de-la-vulgarisation/',
 '/2017/01/07/qu-est-ce-qu-un-scriptorium/',
 '/2016/12/20/quels-étaient-les-moyens-de-fabriquer-de-l-encre-noire-au-moyen-âge/',
 '/2016/12/14/traducteur-de-l-art-de-transposer-sans-dénaturer/',
 '/2016/12/09/de-la-lettre-manuscrite-au-sms/',
 '/2016/12/04/quelle-écriture-était-utilisée-et-transmise-exclusivement-par-des-femmes/',
 '/2016/11/29/papyrologue-éditeur-de-textes-anciens/',
 '/2016/11/26/qu-est-ce-qu-un-alphabet-consonantique/',
 '/2016/11/18/quel-est-le-mythe-à-l-origine-de-l-écriture-chinoise/',
 '/2016/11/15/le-correcteur-au-coeur-des-mots/',
 '/2016/11/09/que-reste-t-il-des-écri

### Titles

In [47]:
for title in soup.findAll('h2', attrs={'class': 'j-blog-headline'}):
    print(title.text)

Ghostwriter, écrivain fantôme, prête-plume... qu'est-ce?
Secrétaire juridique - L'écriture, ce vecteur codé et procédural
Maître Marc Bonnant à l'honneur
Harmonie Keller Matthey, blogueuse: "L'écriture est un art libérateur"
Exposition sur l'alphabet a ka u ku
Rédacteur technique ou l'art de la vulgarisation
Qu'est-ce qu'un scriptorium?
Quels étaient les moyens de fabriquer de l'encre noire au Moyen-Âge?
Traducteur: de l'art de transposer sans dénaturer
De la lettre manuscrite au sms
Quelle écriture était utilisée et transmise exclusivement par des femmes?
Papyrologue: éditeur de textes anciens
Qu'est-ce qu'un alphabet consonantique?
Quel est le mythe à l'origine de l'écriture chinoise?
Le correcteur au coeur des mots
Que reste-t-il des écrits gaulois?
L'expertise en écritures: différencier le vrai du faux
Comment fonctionne l'alphabet braille et comment les aveugles peuvent-ils lire à l'écran ?
Comment a été découverte la première tablette d'écriture rongo-rongo à l'île de Pâques?
Réd

### Dates

In [48]:
for date in soup.findAll('time', attrs={'class': 'datetime'}):
    print(date.text.replace('\n', ' '))

 dim. 12 févr. 2017 
 mer. 01 févr. 2017 
 mer. 25 janv. 2017 
 mar. 17 janv. 2017 
 sam. 14 janv. 2017 
 mer. 11 janv. 2017 
 sam. 07 janv. 2017 
 mar. 20 déc. 2016 
 mer. 14 déc. 2016 
 ven. 09 déc. 2016 
 dim. 04 déc. 2016 
 mar. 29 nov. 2016 
 sam. 26 nov. 2016 
 ven. 18 nov. 2016 
 mar. 15 nov. 2016 
 mer. 09 nov. 2016 
 mer. 02 nov. 2016 
 ven. 28 oct. 2016 
 ven. 21 oct. 2016 
 mar. 18 oct. 2016 


### Summary

In [126]:
for summary in soup.findAll('div', attrs={'class':'j-module n j-text '}):
    txt2 = summary.text.replace("\n   ", "")
    txt3 = txt2[1:len(txt2)-1]
    print(txt3)
    print('-----------------------------')

D'aucuns, ardents défenseurs de la langue française et de ses subtilités, se sont offusqués de l'utilisation du terme ghostwriting dans l'intitulé de nos débats, arguant notamment que celui-ci était incompréhensible pour la plupart des gens, sous-entendu francophones. Cela dit, je ne suis pas certaine que les termes écrivain à gage... 
-----------------------------
Impossible pour elle de s’adresser à sa fiduciaire ou à sa gérance immobilière sans y mettre les formes. Ses courriers sont toujours soignés et habilement rédigés. « Déformation professionnelle », sourit Coralie Bovet. A 29 ans, la Vaudoise a derrière elle près de 10 ans d’expérience en qualité de secrétaire juridique. Et forcément, ça laisse des traces.
-----------------------------
Notre équipe est à pied d'oeuvre et notre comité se réunit à un rythme de plus en plus soutenu pour régler les derniers détails d'organisation... et puis aussi les imprévus. Un peu de stress, certes, mais aussi beaucoup de plaisir... Certains au

## Retrieve all articles
```
{
  "category" : "Actualité - Organisation - Exposition",
  "content" : "......",
  "date" : "05.11.2016", # Article date
  "id" : 0, # Unique
  "intro" : "Pour sa première édition, le Salon International de l’Ecriture aura le plaisir de présenter une exposition sur l’alphabet inventé en 1895 par le roi Ibrahim Njoya, sultan du Royaume Bamoun au Cameroun.",</br>
  "title" : "Exposition sur l'alphabet a ka u ku",
  "last_update" : Datetime.datetime.now(),
}
```

In [144]:
articleCounter = 0
articles_infos = []
articles_html = []
def generate_articles_json(article_url) :
    global articleCounter
    print(articleCounter, ' : ', article_url)
    article_html = requests.get(article_url)
    soup = BeautifulSoup(article_html.content, 'html.parser')
    content = '<!DOCTYPE html><html lang="fr-FR">{0}<body>{1}</body></html>'.format(soup.head, soup.section)
    content_ = content
    
    
    category = soup.find('p', attrs={'class' : 'postmeta'}).text
    article_date = soup.find('time').attrs.get('datetime')
    
    #Intro
    intro = soup.find('div', attrs={'class' : 'j-module n j-text '}).text
    intro2 = intro.replace("\n   ", "")
    intro3 = intro2[1:len(intro2)-1]
    
    title = soup.find('h1', attrs={'class' : 'j-blog-header j-blog-headline'}).text
    
    article_json = {
            'category' : category,
            'date' : article_date,
            'id' : articleCounter,
            'intro' : intro3,
            'title' : title,
            'last_update' : str(datetime.now().date())
            #content = soup.find_all('div', attrs={'class' : 'j-module n j-text '})[1]
        }
#     print(repr(article_json['intro']))
    articleCounter += 1
    articles_html.append(content_)
    articles_infos.append(article_json)

In [145]:
base_url = 'https://www.salonecriture.org'

articleCounter = 0
articles_infos = []
articles_html = []
# first = True
for article_link in all_links:
#     if first:
    generate_articles_json(base_url + article_link)
#         first = False

0  :  https://www.salonecriture.org/2017/02/12/ghostwriter-écrivain-fantôme-prête-plume-qu-est-ce/
1  :  https://www.salonecriture.org/2017/01/28/secrétaire-juridique-l-écriture-ce-vecteur-codé-et-procédural/
2  :  https://www.salonecriture.org/2017/01/25/maître-marc-bonnant-à-l-honneur/
3  :  https://www.salonecriture.org/2017/01/17/harmonie-keller-matthey-blogueuse-l-écriture-est-un-art-libérateur/
4  :  https://www.salonecriture.org/2017/01/14/exposition-sur-l-alphabet-a-ka-u-ku/
5  :  https://www.salonecriture.org/2017/01/11/rédacteur-technique-ou-l-art-de-la-vulgarisation/
6  :  https://www.salonecriture.org/2017/01/07/qu-est-ce-qu-un-scriptorium/
7  :  https://www.salonecriture.org/2016/12/20/quels-étaient-les-moyens-de-fabriquer-de-l-encre-noire-au-moyen-âge/
8  :  https://www.salonecriture.org/2016/12/14/traducteur-de-l-art-de-transposer-sans-dénaturer/
9  :  https://www.salonecriture.org/2016/12/09/de-la-lettre-manuscrite-au-sms/
10  :  https://www.salonecriture.org/2016/12/04

In [146]:
nowStr = datetime.now().strftime("%Y-%m-%d_%Hh%Mmin%S")

exportPathInfos = os.path.join('export', 'articles_' +nowStr + '_infos.json')
exportPathHtml = os.path.join('export', 'articles_' + nowStr + '_html.json')

In [147]:
with open(exportPathInfos, 'w') as f:
        json.dump(articles_infos, f)

In [148]:
with open(exportPathHtml, 'w') as f:
        json.dump(articles_html, f)